# Imports and Setup
----------

- Package Imports

In [3]:
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import seaborn as sb
import pandas as pd
import numpy as np

- Data Imports

In [4]:
df_education = (
   pd.read_csv('D:/Courses/Python-Dash/Interactive+Dashboards+with+Plotly+&+Dash/Course_Materials/Data/states_all.csv')
   .assign(
        exp_per_std = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"],
        math8_above = lambda x: np.where(
            x["AVG_MATH_8_SCORE"] > x["AVG_MATH_8_SCORE"].mean(), 'Above Avg', 'Below Avg'
        )
   ) 
)

df_education.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,exp_per_std,math8_above
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN,3.627221,Below Avg
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN,7.939520,Below Avg
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN,5.050774,Below Avg
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN,3.948044,Below Avg
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN,5.164536,Below Avg


### Dash App Boilerplate

In [ ]:
# top level app interface
app = Dash(__name__)

# layout
app.layout = html.Div([
    "Dropdown Container",
    dcc.Dropdown(
        options = ['US', 'Portugal', 'Albania', 'Russia'],
        value = 'Portugal',
        id = 'input-el'
    ),
    html.Div(id = 'el-output')
    
])

# callback
@app.callback(
    Output('output-el', 'children'), 
    Input('input-el', 'value')
)
def country_picker(country):
    if not country:
        raise PreventUpdate
    return f"The country selected is {country}"

# start server
if __name__ == "__main__":
    app.run(debug = True)

## 1) Dropdown Demo

In [14]:
# top level app interface
app = Dash(__name__)

# layout
app.layout = html.Div([
    dcc.Dropdown(
        #options = df_education["STATE"].unique(),
        options = [
            {
                "label": state.title(),
                "value": state
            } for state in df_education["STATE"].unique()
        ],
        value = 'CALIFORNIA',
        id = 'inputEl-dropdown',
        multi = True
    ),
    dcc.Graph(id = 'outputEl-graph'),
    
])

# callback
@app.callback(
    Output('outputEl-graph', 'figure'), 
    Input('inputEl-dropdown', 'value')
)

def plot_rev_line(state):
    fig = px.line(
        (df_education
        .query("STATE in @state and 1992 < YEAR < 2016")
        .groupby(["YEAR", "STATE"])
        .sum()
        .reset_index()
        ),
        x = "YEAR",
        y = "TOTAL_EXPENDITURE",
        color = "STATE",
        #title = f"Expenditure by {state}"
        title = "Expenditure by Selected States"
    )
    return fig
    

# start server
if __name__ == "__main__":
    app.run(debug = True, mode = "inline")

## 2) Checklists

In [31]:
df_skiResorts = pd.read_csv('D:/Courses/Python-Dash/Interactive+Dashboards+with+Plotly+&+Dash/Course_Materials/Data/European_Ski_Resorts.csv') 
df_skiResorts.head()

,Resort,Country,HighestPoint,LowestPoint,DayPassPriceAdult,BeginnerSlope,IntermediateSlope,DifficultSlope,TotalSlope,Snowparks,NightSki,SurfaceLifts,ChairLifts,GondolaLifts,TotalLifts,LiftCapacity,SnowCannons
0,Alpendorf (Ski amedé),Austria,1980,740,52,30,81,4,115,Yes,No,22,16,11,49,75398,600
1,Soldeu-Pas de la Casa/​Grau Roig/​El Tarter/​C...,Andorra,2640,1710,47,100,77,33,210,Yes,Yes,37,28,7,72,99017,1032
2,Oberau (Wildschönau),Austria,1130,900,30,1,0,1,2,No,No,2,0,0,2,1932,0
3,Dachstein West,Austria,1620,780,42,15,33,3,51,Yes,Yes,25,8,3,36,32938,163
4,Rosa Khutor,Southern Russia,2320,940,22,30,26,21,77,Yes,No,6,11,10,27,49228,450


In [32]:
df_skiResorts.shape

(376, 17)

- Dash app

In [40]:
# top level app interface
app = Dash(__name__)

# layout
app.layout = html.Div([
    html.H2(id = "title"),
    dcc.Checklist(
        id = "inputEl-checklist",
        options = ["Has Snow Park", "Has Night Ski"],
        value = ["Has Snow Park", "Has Night Ski"],
    ),
    dcc.Graph(id = 'outputEl-graph'),
    
])

# callback
@app.callback(
    Output('title', 'children'),
    Output('outputEl-graph', 'figure'), 
    Input('inputEl-checklist', 'value'),
)

def plot_skiMap(features):
    if features == []:
        df = (df_skiResorts
            .groupby("Country", as_index = False)
            .agg({"TotalLifts": "sum"})
            )
    elif len(features) == 2:
        df = (df_skiResorts
            .query("Snowparks == 'Yes' and NightSki == 'Yes'")
            .groupby("Country", as_index = False)
            .agg({"TotalLifts": "sum"})
            )
    elif features == ["Has Snow Park"]:
        df = (df_skiResorts
            .query("Snowparks == 'Yes'")
            .groupby("Country", as_index = False)
            .agg({"TotalLifts": "sum"})            
            )
    else:
       df = (df_skiResorts
            .query("NightSki == 'Yes'")
            .groupby("Country", as_index = False)
            .agg({"TotalLifts": "sum"})            
            )

    fig = px.choropleth(
       df,
       locations = "Country",
       color = "TotalLifts",
       locationmode = "country names",
       scope = "europe",
    ) \
    .update_geos(fitbounds = "locations") \
    .update_layout(margins = {"r": 0,"t": 0,"l": 0,"b": 0})

    title = f"{features}"
    return title, fig

# start server
if __name__ == "__main__":
    app.run(debug = True)